# 07 · Evaluate and Run Ablations

## Purpose

Measure retrieval quality and perform robustness checks on the unified pipeline.

## Inputs

- `outputs/predictions/test.csv` (and optionally dev predictions).
- `data/processed/artifacts_with_split.csv` with gold controls for comparison.

## Outputs

- `eval/tables/metrics.csv` capturing aggregate and per-family metrics.
- Notebook tables/plots summarizing ablation and leak-check findings.

## Steps

1. Compute Top-1, P@k, R@k, Jaccard@k (k ∈ {1,3,5}), MAP, MRR, and per-family breakdowns.
2. Verify partition integrity and surface sample comparisons of gold vs predictions.
3. Re-run the duplicate text hash to ensure zero leakage across splits.
4. Perform an adversarial label shuffle to confirm metrics collapse under label noise.
5. Document additional ablations (e.g., w/o cross-encoder, w/o Auto-K) if time allows.

## Acceptance Checks

- `eval/tables/metrics.csv` is written with the required metrics.
- Leak-check diagnostics (duplicates, partition coverage, random samples) are reported.
- Ablation results and observations are captured in the notebook narrative.